# Neural Networks 

- The following notebook corresponds to the experimentation and implementation of neural networks for the article <insert DOI here>. Data importing, pre-processing and sampling can be performed following the code presented in https://github.com/glarreag/Manu_road
- This Jupyter notebooks contains snippets from code presented in [GEE tutorials](https://developers.google.com/earth-engine/tf_examples)


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())
print(tf.__version__)
device_name = tf.test.gpu_device_name()
device_name

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=dc692ffc8f7ce2e244f1ef2f08119d07f14d5d5aa0b6ab22c26c1a8f802bfcf4
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.2 GB  | Proc size: 1.1 GB
GPU RAM Free: 15927MB | Used: 353MB | Util   2% | Total 16280MB


In [ ]:
!pip install -U PyDrive

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import itertools
# import tensorflow as tf

import scipy.cluster.hierarchy as hac


from pprint import pprint
from pathlib import Path

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)
import warnings
from IPython.display import display, Markdown


# from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler, RobustScaler
# from sklearn.decomposition import PCA
# from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor
# from sklearn.model_selection import cross_val_score, GridSearchCV
# from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import GaussianNB
# from sklearn.ensemble import RandomForestClassifier

#Import authentication libraries
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

%matplotlib inline
%config IPCompleter.greedy=True
warnings.filterwarnings('ignore')
sns.set()
%matplotlib inline

In [ ]:
#Authenticating google account to import and export from drive and GCP
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


## Data import

Training data can be imported directly from a Google Drive folder.

If the html is:
https://drive.google.com/drive/folders/1TFe4wT4pTcTwVs0bJwtvikv0BmbTxuBa

the file id is: 
1TFe4wT4pTcTwVs0bJwtvikv0BmbTxuBa


In [ ]:
folder_id= "<your folder id>" # https://drive.google.com/drive/folders/1TFe4wT4pTcTwVs0bJwtvikv0BmbTxuBa
# folder_id='1uXkcRXuQ5fFG0S8mLW67ewj_jsqAb_Ff'
file_list = drive.ListFile({'q': "'{0}' in parents and trashed=false".format(folder_id)}).GetList()

# Download all files to the VM
for file1 in file_list:
  file_id=file1['id']
  file_title = file1['title']
  if '.csv' in file_title: #import only csv
    drive.CreateFile({'id': file_id}).GetContentFile(file_title)

In [ ]:
#check that the files are in this VM
!ls

## Exploring variables

In [ ]:
#Data sample from 2004 - 2017 period
#WITHOUT LAN LON
file_name = "<your_file_name_here.csv>"

# discard unused variables, the following variables are always included by GEE exporting 
training = pd.read_csv(file_name, sep=',').drop(['system:index', '.geo','cluster_mean', "random"],axis=1)  
validation = pd.read_csv(file_name, sep= ",").drop(['system:index', '.geo','cluster_mean', "random"],axis=1)


Data for prediction was exported following the script [link to earth engine exporting scrpit] as TFRecord files. During task preparation it is important to specify the patch_size and the max_size parameteres. The complete image to predict is divided in patches of dimensions  [patch_size, patch_size]. The TFRecord file is divided in various TFRecord files of size=max_size.

An aditional "mixer" json file is exported. This file is necessary to map the predicted file into the Earth Engine plataform

## Model construction


Fundamental layers definition


In [ ]:
# Batch Normalization func
def batch_norm(x, mode = None, train=True, name="batch_norm", training=False): # using batchnormalization training (moving means) 
  with tf.variable_scope(name):
    normie=tf.layers.batch_normalization(x, name=name,momentum=0.90, training=training)
    return normie
    
#Linear activation
def dense(input_, output_size, name=None, stddev=0.02, bias_start=0.0, regularization='none' ,activation='relu'):
    shape = input_.get_shape().as_list()
    
    if regularization == 'l2': #If regularization is wanted. Only expects 'l2' regularization
      with tf.variable_scope(name):
          matrix = tf.get_variable("Weights", [shape[1], output_size], tf.float32,
                                   tf.initializers.glorot_normal(seed=0), regularizer=tf.keras.regularizers.l2(l=0.005))
          bias = tf.get_variable("bias", [output_size],
              initializer=tf.constant_initializer(bias_start))
          linear = tf.matmul(input_, matrix) + bias
          if activation == 'relu':
            return tf.nn.relu(linear, name='activation_relu')
          if activation == 'sigmoid':
            return tf.nn.sigmoid(linear, name='activation_relu')
          if activation == 'logits':
            return linear
          
    if regularization == 'none':
      with tf.variable_scope(name):
          matrix = tf.get_variable("Weights", [shape[1], output_size], tf.float32,
                                   tf.initializers.glorot_normal(seed=0))
          bias = tf.get_variable("bias", [output_size],
              initializer=tf.constant_initializer(bias_start))
          linear = tf.matmul(input_, matrix) + bias
          if activation == 'relu':
            return tf.nn.relu(linear, name='activation_relu')
          if activation == 'sigmoid':
            return tf.nn.sigmoid(linear, name='activation_relu')
          if activation == 'logits':
            return linear      
        
#Dropout      
def dropout(input_,name='dropout', training=False, mode = None):
  dropped = tf.layers.dropout(
                inputs=input_,
                rate=0.1,
                noise_shape=None,
                seed=None,
                training = training,
                name=name)
  return dropped


Different network architectures can be defined in this cell:

In [ ]:


def net_0(x,training=False): 
  
  d1= dense(x, 256,name='d1')
  d2= dense(d1,128,name ='d2')
  d3 = dense(d2,64, name = 'd3')
  d4 = dense(d3,32, name = 'd4')
  d5 = dense(d4,16, name = 'd5')
  logits = dense(d5,1, name = 'logits', activation='logits')[:,0]
  predicted = dense(d5,1, name = 'predicted', activation='sigmoid')[:,0]
  return logits, predicted

def net_0_reg(x, mode='train',training=False): 
  
  d1= dense(x, 256,name='d1',regularization='l2')
  d2= dense(d1,128,name ='d2',regularization='l2')
  d3 = dense(d2,64, name = 'd3',regularization='l2')
  d4 = dense(d3,32, name = 'd4',regularization='l2')
  d5 = dense(d4,16, name = 'd5',regularization='l2')
  logits = dense(d5,1, name = 'logits', activation='logits')[:,0]
  predicted = dense(d5,1, name = 'predicted', activation='sigmoid')[:,0]
  return logits, predicted

def net_0_1(x, training=False):
  
  d1= dense(x, 1024,name='d1')
  d2= dense(d1,1024,name ='d2')
  d3 = dense(d2,512, name = 'd3')
  d4 = dense(d3,256, name = 'd4')
  d5 = dense(d4,128, name = 'd5')
  d6 = dense(d5,56, name = 'd6') 
  dp0= dropout(d6,name='dp0')
  logits = dense(dp0,1, name = 'logits', activation='logits')[:,0]
  predicted = dense(d6,1, name = 'predicted', activation='sigmoid')[:,0]
  return logits, predicted



def net_1_reg(x,training=False): 
  
  d1= dense(x, 512,name='d1',regularization='l2')
  bn1 = batch_norm(d1,name='bn1',training=training)
  dp0= dropout(bn1,name='dp0',training=training)
  d2= dense(dp0,128,name ='d2',regularization='l2')
  bn2 = batch_norm(d2,name='bn2',training=training)
  dp1= dropout(bn2,name='dp1',training=training)
  d3 = dense(dp1,64, name = 'd3',regularization='l2')
  bn3 = batch_norm(d3,name='bn3',training=training)
  dp2= dropout(bn3,name='dp2',training=training)
  d4 = dense(dp2,32, name = 'd4',regularization='l2')
  bn4 = batch_norm(d4,name='bn4',training=training)
  dp3= dropout(bn4,name='dp3',training=training)
  d5 = dense(dp3,16, name = 'd5',regularization='l2')
  bn5 = batch_norm(d5,name='bn5',training=training)
  logits = dense(bn5,1, name = 'logits', activation='logits')[:,0]
  predicted = dense(bn5,1, name = 'predicted', activation='sigmoid')[:,0]
  return logits, predicted



def nn_6_1024(x,mode='train',training=False):
  
  d1= dense(x, 1024,name='d1')
  bn1 = batch_norm(d1,name='bn1',mode=mode)
  d2= dense(bn1,512,name ='d2')
  dp1= dropout(d2,name='dp1',mode=mode)
  d3 = dense(dp1,256, name = 'd3')
  bn3 = batch_norm(d3,name='bn3',mode=mode)
  d4 = dense(bn3,128, name = 'd4')
  dp3= dropout(d4,name='dp3',mode=mode)
  d5 = dense(dp3,64, name = 'd5')
  dp5= dropout(d5,name='dp5',mode=mode)
  d6 = dense(dp5,32, name = 'd6')
  logits = dense(d6,1, name = 'logits', activation='logits')[:,0]
  predicted = dense(d6,1, name = 'predicted', activation='sigmoid')[:,0]
  return logits, predicted

def nn_6_1024_reg(x,training=False):
  
  d1= dense(x, 1024,name='d1',regularization='l2')
  bn1 = batch_norm(d1,name='bn1',training=training)
  d2= dense(bn1,512,name ='d2',regularization='l2')
  dp1= dropout(d2,name='dp1',training=training)
  d3 = dense(dp1,256, name = 'd3',regularization='l2')
  bn3 = batch_norm(d3,name='bn3',training=training)
  d4 = dense(bn3,128, name = 'd4',regularization='l2')
  dp3= dropout(d4,name='dp3',training=training)
  d5 = dense(dp3,64, name = 'd5',regularization='l2')
  dp5= dropout(d5,name='dp5',training=training)
  d6 = dense(dp5,32, name = 'd6',regularization='l2')
  logits = dense(d6,1, name = 'logits', activation='logits')[:,0]
  predicted = dense(d6,1, name = 'predicted', activation='sigmoid')[:,0]
  return logits, predicted

def nn_6_5000_reg(x,training=False): 
  
  d1= dense(x, 2000,name='d1',regularization='l2')
  bn1 = batch_norm(d1,name='bn1',training=training)
  d2= dense(bn1,1000,name ='d2',regularization='l2')
  dp1= dropout(d2,name='dp1',training=training)
  d3 = dense(dp1,500, name = 'd3',regularization='l2')
  d4 = dense(d3,600, name = 'd4',regularization='l2')
  dp3= dropout(d4,name='dp3',training=training)
  d5 = dense(dp3,100, name = 'd5',regularization='l2')
  d6 = dense(d5,30, name = 'd6',regularization='l2')
  logits = dense(d6,1, name = 'logits', activation='logits')[:,0]
  predicted = dense(d6,1, name = 'predicted', activation='sigmoid')[:,0]
  return logits, predicted


def nn_7_deep(x,training=False):
  
  d1= dense(x, 2048,name='d1',regularization='l2')
  bn1 = batch_norm(d1,name='bn1',training=training)
  d1_1= dense(bn1, 2048,name='d1_1',regularization='l2')
  d2= dense(d1_1,1024,name ='d2',regularization='l2')
  d3 = dense(d2,1024, name = 'd3',regularization='l2')
  bn3 = batch_norm(d3,name='bn3',training=training)
  d4 = dense(bn3,512, name = 'd4',regularization='l2')
  d5 = dense(d4,264, name = 'd5',regularization='l2')
  d6_1 = dense(d5,120, name = 'd6_1',regularization='l2')
  d6 = dense(d6_1,32, name = 'd6',regularization='l2')
  dp5= dropout(d6,name='dp5',training=training)
  logits = dense(dp5,1, name = 'logits', activation='logits')[:,0]
  predicted = dense(dp5,1, name = 'predicted', activation='sigmoid')[:,0]

  return logits, predicted
  
def net_2(x,training=False): 
  
  d1= dense(x, 60,name='d1')
  d2= dense(d1,15,name ='d2')
  logits = dense(d2,1, name = 'logits', activation='logits')[:,0]
  predicted = dense(d2,1, name = 'predicted', activation='sigmoid')[:,0]
  return logits, predicted

def reg_0(x):
  
  d1= dense(x, 120,name='d1')
  bn1 = batch_norm(d1,name='bn1')
  d2= dense(d1,60,name ='d2')
  logits = dense(d2,1, name = 'logits', activation='logits')[:,0]
  predicted = dense(d2,1, name = 'predicted', activation='sigmoid')[:,0]
  return logits, predicted


## Model definition


The main layers are defined using the low level API of Tensorflow.
Three main modes have been stablished:

1.   **'train'**: Uses the csv training data imported as panda dataframe as input.
2.   **'predict'**: Uses any panda dataframe data as input. If labels are included, the model returns the accuracy and the predicted labels. If no labels are included, the model only returns predicted labels
3.   **'tfrecord'**: This mode is designed exclusively to make predictions from a TFRecord file, as exported from Earth Engine. The purpose of this mode is to predict large scale territories and returns a TFRecord file containing the predicted probabilities of each pixel, ready for Earth Engine ingestion.

In [ ]:
predicted_prob = []
predicted_int = []
correct_bool = []

def RUN(sess,X=None,y=None,X_val = None, y_val = None, X_test = None, y_test= None,PATCH_WIDTH=256, PATCH_HEIGHT=256,batch_size=256, epoch=100, learning_rate=0.001, 
        training=False,mode='train',name='model_1',test_file=None,mean_=None,scale_=None,new=False,model_folder=None, model_func=None):
  
  global_step = tf.Variable(0, name='global_step', trainable=False)
  increment_global_step = tf.assign_add(global_step, 1, name = 'increment_global_step')
  
  if mode == 'train':
    training=True
  
  if mode == 'tfrecord':  
    X_tensor = predict_input_fn(test_file,mean_,scale_) #this ensure to have a tensor with values as X_tensor
  else:
    X_tensor= tf.placeholder(tf.float32, shape=(None,X.shape[1]),
                                    name='X_tensor')
  y_tensor= tf.placeholder(tf.float32, shape=(None),
                                    name='y_tensor')
    
  def model(x,y=None,name='model_1', reg_loss=0, training=False):
    with tf.variable_scope(name, reuse= tf.AUTO_REUSE):
      logits, predicted = model_func(x,training=training)  
    return logits, predicted 

  # with tf.variable_scope(name, reuse= tf.AUTO_REUSE):  
  if mode =='train':
    
    logits, predicted = model(X_tensor, y_tensor,name=name, training=training)  

    print(tf.global_variables())
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(_sentinel=None, 
            labels=y_tensor, logits=logits, name='loss'))
    
    reg_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    reg_loss = tf.reduce_sum(reg_loss)
    tot_loss = tf.add(loss,reg_loss)
    opt = tf.train.AdamOptimizer(learning_rate)
    # opt = tf.train.GradientDescentOptimizer(learning_rate)
    
    
    t_vars = tf.trainable_variables()
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
      model_opt=opt.minimize(tot_loss, var_list=t_vars)
      
    start_time = time.time()

    print(tf.global_variables())
    if new == True:
      tf.global_variables_initializer().run()
    else:
      saver=tf.train.Saver()
      ckpt = tf.train.get_checkpoint_state(model_folder_root+model_folder)
      saver.restore(sess, ckpt.model_checkpoint_path)
      

    print(tf.global_variables())
    
    for epoch in range(epoch):
      X, y = shuffle(X,y)
      batch_idxs = len(X) // batch_size
      for idx in range(0, batch_idxs): 
        batch_X = X[idx*batch_size:(idx+1)*batch_size]
        batch_y = y[idx*batch_size:(idx+1)*batch_size]
        
        sess.run(model_opt, feed_dict={X_tensor: batch_X, y_tensor: batch_y})
      

      loss_print=tf.reduce_mean(loss)
      classified = tf.to_int32(predicted > 0.5)
      correct_train = tf.equal(classified, y_train.values)
      correct_val = tf.equal(classified, y_val.values)
      correct_test = tf.equal(classified, y_test.values)
      
      train_loss = loss_print.eval({X_tensor: X_train, y_tensor: y_train})     
      val_loss = loss_print.eval({X_tensor: X_val, y_tensor: y_val})
      
      train_acc = tf.reduce_mean(tf.cast(correct_train, tf.float32)).eval({X_tensor: X_train, y_tensor: y_train})
      val_acc = tf.reduce_mean(tf.cast(correct_val, tf.float32)).eval({X_tensor: X_val, y_tensor: y_val})
      test_acc = tf.reduce_mean(tf.cast(correct_test, tf.float32)).eval({X_tensor: X_test, y_tensor: y_test})
      
      sess.run(increment_global_step)
      new_epoch = global_step.eval()
      print("Epoch: [%2d] time: %4.4f, train_loss: %.8f, val_loss: %.8f, train_acc: %.8f, val_acc: %.8f, test_acc: %.8f" \
                  % (new_epoch, time.time() - start_time, train_loss, val_loss,train_acc,val_acc,test_acc))
    
    saver=tf.train.Saver()
    saver.save(sess, model_folder_root+model_folder+'model.ckpt', global_step=global_step)
      

  if mode =='predict':
    
    print('aquitoy_func')
    print(tf.global_variables())
    
    logits, predicted = model(X_tensor, y_tensor,name='model_1',training=training)  

    
    saver=tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state(model_folder_root+model_folder)
    saver.restore(sess, ckpt.model_checkpoint_path)

    print('aquitoy_predict')
    print(tf.global_variables())
    predicted_value=predicted.eval({X_tensor: X, y_tensor: y})
  
    print(predicted_value)
    
    predicted_prob.append(predicted_value)
    classified = tf.to_int32(predicted > 0.5)
    
    classified_value = classified.eval({X_tensor: X, y_tensor: y})
    predicted_int.append(classified_value)
    
    correct_prediction = tf.equal(classified, y)

    correct_value = correct_prediction.eval({X_tensor: X, y_tensor: y})

    correct_bool.append(correct_value)

    print(correct_value)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)).eval({X_tensor: X, y_tensor: y})
    print('DA A-Q-RA-C', accuracy)

  if mode =='tfrecord':
    ##Makes an example_proto taking a predicted value as input
    def make_example(pred):
        return tf.train.Example(
          features=tf.train.Features(
            feature={
              'probability': tf.train.Feature(
                  float_list=tf.train.FloatList(
                      value=pred))
            }
          )
        )

  
    logits, predicted = model(X_tensor,name='model_1',training=training)
    
    
    saver=tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state(model_folder_root+model_folder)
    saver.restore(sess, ckpt.model_checkpoint_path)

    
    patch = [[]]
    curPatch = 1

    writer = tf.python_io.TFRecordWriter(outputImageFile)
    still_writing= True
    start_time = time.time()     

    while still_writing: #We predict only until reaching the patch size (batch size) so we can save it and export it as tfrecord again

        
      try:
        value = predicted.eval()[0]
        patch[0].append(value)


        if (len(patch[0]) == PATCH_WIDTH * PATCH_HEIGHT):
          print('Done with patch ' + str(curPatch) + '...'+' Elapsed time: {:.2f}'.format(time.time()-start_time))

          # Create an example
          example=make_example(patch[0])
                
          writer.write(example.SerializeToString())
          patch = [[]]
          curPatch += 1
          start_time = time.time() 

      except: 
          still_writing=False
    
    writer.close()
    print('bug')


## Training

1.   Introduce a folder to store checkpoint objects.
2.   GCP buckets can be used too
3.   Multiple models, number of epochs, and training samples can be evaluated

In [ ]:
your_folder_root = "<insert_your_folder>" 
model_folder_root = your_folder_root
epochs = [100, 300, 300, 700, 700]
models = [net_0, net_0_reg,net_2,net_0_1,nn_6_5000_reg] #example
datasets_names = ["04_17"] #example
datasets_t = [training]
datasets_v = [validation] 
model_folder =  ['net_0_noXY/', 'net_0_reg_noXY/','net_2_noXY/', 'net_0_1_noXY/','nn_6_5000_reg_noXY/'] #example
model_names =  ['net_0', 'net_0_reg','net_2', 'net_0_1','nn_6_5000_reg'] #example

In [ ]:
for training_set, validation_set, db_name in zip(datasets_t, datasets_v, datasets_names):
  
  X_trainval = training_set.drop('lossyear', axis=1)
  y_trainval = training_set['lossyear']

  X_test = validation_set.drop('lossyear', axis=1)
  y_test = validation_set['lossyear']

  scaler = preprocessing.RobustScaler().fit(X_trainval) #do not forget to use the same scaler object to predict

  X_trainval = scaler.transform(X_trainval)
  X_test = scaler.transform(X_test)

  X_train, X_val, y_train, y_val = train_test_split(X_trainval,y_trainval, test_size=0.3,random_state=0)

  for model, folder, epoch, model_name in zip(models,model_folder, epochs, model_names):

    folder = db_name + "/" + folder

    g = tf.Graph()
    with g.as_default():
      with tf.Session() as sess:
        RUN(X=X_train,y=y_train,X_val=X_val, y_val= y_val,X_test =X_test,y_test=y_test, sess=sess,epoch=epoch,batch_size = 250, learning_rate=0.0001, new=True, model_folder=folder, model_func=model)
    
   

## Predicting the validation set

In [ ]:
# repeating the normalization process to ensure the same output (use the same random_state)
for training_set, validation_set, db_name in zip(datasets_t, datasets_v, datasets_names):
  
  X_trainval = training_set.drop('lossyear', axis=1)
  y_trainval = training_set['lossyear']

  X_test = validation_set.drop('lossyear', axis=1)
  y_test = validation_set['lossyear']

  scaler = preprocessing.RobustScaler().fit(X_trainval)

  X_trainval = scaler.transform(X_trainval)
  X_test = scaler.transform(X_test)

  X_train, X_val, y_train, y_val = train_test_split(X_trainval,y_trainval, test_size=0.3,random_state=0)

  for model, folder, epoch, model_name in zip(models,model_folder, epochs, model_names):

    folder = db_name + "/" + folder

    predicted_prob = []
    g = tf.Graph()
    with g.as_default():
      with tf.Session() as sess:
        RUN(X=X_test,y=y_test.values,sess=sess,mode='predict', model_folder=folder,model_func=model)

    for_dataframe= {"cluster":y_test, "classification":predicted_prob[0]}
    df = pd.DataFrame(for_dataframe)
    file_name = "predicted_"+db_name+"_"+model_name+".csv" 
    df.to_csv(file_name, sep=',', encoding='utf-8') # a csv file is created with the probability value 


    ## uncomment to export CSV to Google Drive folder
    # file = drive.CreateFile({'parents':[{u'id': folder_id}]})
    # file.SetContentFile(file_name)
    # file.Upload() 

## Image prediction


1.   The image to predict is in tfrecord format, exported from GEE (see data_ingestion.js)
2.   A parsing function is defined. This function also normalize the data using the scaling parameters calculated during training.
3. Tfrecord files are stored in a Google Cloud Storage bucket.
4. The predicted imaged needs to be imported to GEE as a tfrecord. An authentication step is required.



Creating tfrecord parsing function

In [ ]:
# Names of the features.
bands = [
  "altitude",
  #  "lossyear",
   "latitude",
  "distance_second",
  "slope",
  "landform",
  "distance_parks",
  # "carbon_density",
  "distance_first",
  # "cluster_mean",
  "distance_villages",
  "distance_buffer",
  "distance_third",
  "distance_any"
  "longitude"
]

bands.sort() #to match the order of exported variables
featureNames = list(bands)

PATCH_WIDTH = 256
PATCH_HEIGHT = 256
PATCH_DIMENSIONS_FLAT = [PATCH_WIDTH * PATCH_HEIGHT, 1] #calculates the number of pixel in each patch

# Feature columns
columns = [
  tf.FixedLenFeature(shape=PATCH_DIMENSIONS_FLAT, dtype=tf.float32) for k in featureNames #Creates a feature for each band 
                                                                                          
]

featuresDict = dict(zip(featureNames, columns)) 

#This function parses the data and normalizes it before predicting 
def predict_input_fn(fileNames,mean_,scale_): #Se ha entrenado normalizando los datos con los datos de entrenamiento
                                              
  
  driveDataset = tf.data.TFRecordDataset(fileNames, compression_type='GZIP')  # Note that you can make one dataset from many files by specifying a list.
  
  #Parsing function
  def parse(example_proto): 
    parsed_features = tf.parse_single_example(example_proto, featuresDict) #Parses tfrecord and returns dict-like object
    
    #to normalize, we need to have tensors: 
    altitude=tf.cast(parsed_features['altitude'], tf.float32),
    distance_any=tf.cast(parsed_features['distance_any'], tf.float32),
    distance_buffer=tf.cast(parsed_features['distance_buffer'], tf.float32),
    distance_second=tf.cast(parsed_features['distance_second'], tf.float32),
    distance_first=tf.cast(parsed_features['distance_first'], tf.float32),
    distance_parks=tf.cast(parsed_features['distance_parks'], tf.float32),
    distance_third=tf.cast(parsed_features['distance_third'], tf.float32),
    distance_villages=tf.cast(parsed_features['distance_villages'], tf.float32),
    slope=tf.cast(parsed_features['slope'], tf.float32),
    landform=tf.cast(parsed_features['landform'], tf.float32),
    latitude=tf.cast(parsed_features['latitude'], tf.float32),
    longitude=tf.cast(parsed_features['longitude'], tf.float32),
    

    #the order matters, be careful with the order of bands
    data = tf.concat([altitude,distance_any,distance_buffer,distance_first,
              distance_parks,distance_second,distance_third,distance_villages,landform,slope, latitude, longitude],axis=0)
    data = tf.transpose(data, [1, 0, 2])
    return data

  def normalize(value,mean_,scale_): 
    #inputs: el tensor sin normalizar, el vector de media y el vector de desviación est.
    
    mean = tf.convert_to_tensor(mean_, dtype=tf.float32) 
    mean = tf.transpose(tf.reshape(mean, [12,1]),[1,0])

    scale=tf.convert_to_tensor(scale_, dtype=tf.float32) 
    scale= tf.transpose(tf.reshape(scale, [12,1]),[1,0])

    new = tf.divide(tf.subtract(value,mean),scale) # normalizing
    return new
  
  parsedDataset = driveDataset.map(parse, num_parallel_calls=5)
  parsedDataset = parsedDataset.flat_map(lambda features: tf.data.Dataset.from_tensor_slices(features))  
  # parsedDataset = parsedDataset.prefetch(1)
  # parsedDataset = parsedDataset.cache()
  iterator = parsedDataset.make_one_shot_iterator().get_next()
  transposed =tf.transpose(iterator, [1,0])
  tensor = normalize(transposed,mean_,scale_)
  return tensor


In [ ]:
#Identify tfrecord file 
files = !gsutil ls gs://roads_and_stuff/
test_file=list(filter(lambda x: '.gz' in x, files))
test_file #several TFRecord files conform the complete image

In [ ]:
mixer_file = list(filter(lambda x: '.json' in x and hint in x, files))
mixer_file[0]
#Counting the number of patches contained in each TFRecord file
options = tf.python_io.TFRecordOptions('GZIP')
for i in test_file:
  size = sum(1 for _ in tf.python_io.tf_record_iterator(i, options=options))
  print("The number of patches in " + i + ' is:' , size)

Creates a predictied tfrecord image file using every model defined in previous cells

In [ ]:
baseName = '<your_folder>'
model_folder_root = your_folder_root

In [ ]:
for training_set, validation_set, db_name in zip(datasets_t, datasets_v, datasets_names):
  
  X_trainval = training_set.drop('lossyear', axis=1)
  y_trainval = training_set['lossyear']

  X_test = validation_set.drop('lossyear', axis=1)
  y_test = validation_set['lossyear']

  scaler = preprocessing.RobustScaler().fit(X_trainval)

  X_trainval = scaler.transform(X_trainval)
  X_test = scaler.transform(X_test)

  X_train, X_val, y_train, y_val = train_test_split(X_trainval,y_trainval, test_size=0.3,random_state=0)

  for model, folder, epoch, model_name in zip(models,model_folder, epochs, model_names):

    folder = db_name + "/" + folder
    outputImageFile = baseName + 'predictions_'+db_name+'_'+model_name+'.tfrecord'
    outputJsonFile = baseName + 'test_predictions.json'

    g = tf.Graph()
    with g.as_default():
      with tf.Session() as sess:
        sess.run(RUN(sess=sess,mode='tfrecord',test_file=test_file,model_folder=folder,model_func = model ,mean_=scaler.center_,scale_=scaler.scale_))


In [ ]:
# Authenticate
!pip install earthengine-api
!earthengine authenticate --quiet

In [ ]:
#insert your folder information
gee_folder = "users/<your_user_name>/<your_folder_name>/"

In [ ]:
for training_set, validation_set, db_name in zip(datasets_t, datasets_v, datasets_names):
  
  X_trainval = training_set.drop('lossyear', axis=1)
  y_trainval = training_set['lossyear']

  X_test = validation_set.drop('lossyear', axis=1)
  y_test = validation_set['lossyear']

  scaler = preprocessing.RobustScaler().fit(X_trainval)

  X_trainval = scaler.transform(X_trainval)
  X_test = scaler.transform(X_test)

  X_train, X_val, y_train, y_val = train_test_split(X_trainval,y_trainval, test_size=0.3,random_state=0)

  for model, folder, epoch, model_name in zip(models,model_folder, epochs, model_names):

    folder = db_name + "/" + folder

    outputImageFile = baseName + 'predictions_'+db_name+'_'+model_name+'.tfrecord'
    outputJsonFile = baseName + 'test_predictions.json'

    outputAssetID = gee_folder+f'{db_name}_{model_name}' 

    !earthengine upload image --asset_id={outputAssetID} {outputImageFile} {mixer_file[0]}
    print(f"done with {model_name} of {db_name}")